In [46]:
# Importing
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
import numpy as np
from pycaret.regression import *


df = pd.read_csv(
    "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv"
)

crime_rate = pd.read_csv(
    "/home/nathan/Documents/School/cse450/module-03/crime_rate.csv"
)

In [47]:
# Add crime rates from nearest city
df = pd.merge(df, crime_rate, on="zipcode", how="left")

In [48]:
# Convert 'date' to datetime and extract year and month
df["date"] = pd.to_datetime(df["date"])
df["year_of_sale"] = df["date"].dt.year
df["month_of_sale"] = df["date"].dt.month
df.drop("date", axis=1, inplace=True)  # Drop the original date column

# Add age of house
df["age"] = 2015 - df["yr_built"]
# Add binary has basement column
df["has_basement"] = (df["sqft_basement"] > 0).astype(int)

In [49]:
numeric_features = [
    "bedrooms",
    "bathrooms",
    "sqft_living",
    "sqft_lot",
    "floors",
    "sqft_above",
    "sqft_basement",
    "yr_built",
    "yr_renovated",
    "lat",
    "long",
    "sqft_living15",
    "sqft_lot15",
]
categorical_features = ["waterfront", "view", "condition", "grade", "zipcode"]

# df = pd.get_dummies(df, columns=categorical_features, drop_first=True)

In [50]:
X = df.drop("price", axis=1)
y = df[["price"]]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=6)

In [51]:
model = LGBMRegressor(n_jobs=-1, random_state=7844)


model.fit(X_train, y_train)

predictions = model.predict(X_test)


# Calculate RMSE
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

r2 = r2_score(y_test, predictions)

print(f"\n\nRMSE: {rmse}")
print(f"\n\nR^2: {r2}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.033833 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2870
[LightGBM] [Info] Number of data points in the train set: 18000, number of used features: 107
[LightGBM] [Info] Start training from score 539596.451778




RMSE: 111798.3823131582


R^2: 0.9118007537402109


In [52]:
mini = pd.read_csv(
    "https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test_mini.csv"
)




In [53]:
# Convert 'date' to datetime and extract year and month
mini["date"] = pd.to_datetime(mini["date"])
mini["year_of_sale"] = mini["date"].dt.year
mini["month_of_sale"] = mini["date"].dt.month
mini.drop("date", axis=1, inplace=True)  # Drop the original date column

mini["has_basement"] = (mini["sqft_basement"] > 0).astype(int)

# Fit and transform the numeric features
mini[numeric_features] = scaler.fit_transform(mini[numeric_features])

# mini = pd.get_dummies(mini, columns=categorical_features, drop_first=True)

NameError: name 'scaler' is not defined

In [ ]:
mini.columns

Index(['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'year_of_sale', 'month_of_sale',
       'has_basement'],
      dtype='object')

In [ ]:
predictions = model.predict(mini)
df = pd.DataFrame(predictions)
df.to_csv("team3-module3-predictions.csv", index=False, header=["price"])

ValueError: Number of features of the model must match the input. Model n_features_ is 25 and input n_features is 22